<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

<div class="alert alert-block" style="background-color: #c7b8d6; padding: 10px">
    <p style="color: #222222">
        <b>Note:</b>
        <br>
        This Jupyter notebook uses hardware features of the Zynq UltraScale+ RFSoC device. Therefore, the notebook cells will only execute successfully on an RFSoC platform.
    </p>
</div>

# Notebook Set C

---

## 01 - Exploring the Spectrum
In this notebook, we will explore the radio spectrum using the Zynq UltraScale+ RFSoC device. Initially, we will prepare the RFSoC for receiving and visualising the radio spectrum. Then, we will investigate several radio frequency bands including FM radio, Digital Audio Broadcasting, digital TV, and mobile communications. Finally, spectrum regulation will be briefly reviewed.

## Table of Contents
* [1. Introduction](#introduction)
    * [1.1. Hardware Setup](#hardware-setup)
    * [1.2. Software Setup](#software-setup)
* [2. RFSoC Spectral Analysis](#rfsoc-spectral-analysis)
* [3. Exploring the Spectrum](#exploring-the-spectrum)
    * [3.1. Frequency Modulated Radio](#frequency-modulated-radio)
    * [3.2. Digital Audio Broadcast](#digital-audio-broadcast)
    * [3.3. Ultra-High Frequency Digital Television](#uhf-digital-tv)
    * [3.4. Spectrum Access for Mobile Communications](#spectrum-mobile)
* [4. Spectrum Regulation](#spectrum-regulation)
* [5. Conclusion](#conclusion)

## References
* [1] - [NooElec, “Nooelec RaTLSnake M6 v2 - Premium 3-Antenna Bundle for SMA-Input SDRs”, webpage.](https://www.nooelec.com/store/ratlsnake-m6.html)
* [2] - [StrathSDR, "RFSoC Spectrum Analyser Module (RFSoC-SAM)", GitHub Repository.](https://github.com/strath-sdr/rfsoc_sam)
* [3] - [OFCOM, "UK Frequency Allocation Table (FAT)", webpage.](http://static.ofcom.org.uk/static/spectrum/fat.html)
* [4] - [FCC, "Federal Communications Commission", webpage.](https://www.fcc.gov/)

## Revision
* **v1.0** | 22/05/23 | *First Revision*

---


## 1. Introduction
Acquiring the radio spectrum requires an antenna and spectrum analyser design. We will begin by setting up your RFSoC platform to begin receiving the radio spectrum with an antenna. Then, we will use an open-source spectrum analyser design for RFSoC to explore the radio spectrum.

### 1.1. Hardware Setup <a class="anchor" id="hardware-setup"></a>
The ZCU208 development boards require the SMA and clock connections to be setup correctly. We will begin with connecting the clocks from the Clk104 card to the development board. Then connect the ADCs and DACs to the HW_XM_655 card's RF baluns.

Before you begin with the setup below, ensure that the Clk104 card and HW_XM_655 card is securely connected to the ZCU208 development board.

#### CLK104 Setup <a class="anchor" id="clk104_setup"></a>
On the Clk104 board, make the following connections as shown in [Figure 1](#fig-1).

<a class="anchor" id="fig-4"></a>
<figure>
<img src='images/clk104_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 1: Setup of the clk104 add-on board on the ZCU208.</b></figcaption>
</figure>

Ensure that the clocks above are connected as follows:
* DAC_229_CLK to DAC_RFCLK_B (P & N).
* ADC_225_CLK to ADC_RFCLK_B (P & N).

If you do not make the correct connection, the demonstration will not operate correctly.

#### ADC Setup <a class="anchor" id="adc_setup"></a>
The next step is to connect the ADCs to the RF Baluns on the HW_XM_655 card. See the image below in [Figure 2](#fig-2) on how to make these connections successfully. You will need the rectangle connector that contains a strip of wires with SMAs attached to make this connection. This connector is known as the Carlisle SMA 8 Cable Assemblies in the kit contents page supplied with your development board.

<a class="anchor" id="fig-2"></a>
<figure>
<img src='images/zcu208_adc_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 2: Setting-up the ADC on the HW_XM_655 card.</b></figcaption>
</figure>

Ensure that the ADC connections are setup as follows:
* JHC5 P0_224 to J61
* JHC5 N01_224 to J63
* JHC5 P2_224 to J2
* JHC5 N23_224 to J6
* JHC5 P0_225 to J1
* JHC5 N01_225 to J5
* JHC5 P2_225 to J18
* JHC5 N23_225 to J16

This step is a little difficult as there are many small wires. Please take your time setting up the ADCs to prevent errors.

#### DAC Setup <a class="anchor" id="dac_setup"></a>
The DACs are connected to the RF Baluns on the HW_XM_655 card using the connections shown in [Figure 3](#fig-3) below.

<a class="anchor" id="fig-3"></a>
<figure>
<img src='images/zcu208_dac_setup.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 3: Setting-up the DAC on the HW_XM_655 card.</b></figcaption>
</figure>

Ensure that the DAC connections are setup as follows:
* JHC3 P0_230 to J64
* JHC3 N0_230 to J66
* JHC3 P2_230 to J28
* JHC3 N2_230 to J30
* JHC3 P0_231 to J26
* JHC3 N0_231 to J29
* JHC3 P2_231 to J39
* JHC3 N2_231 to J37

Take your time with this step to prevent errors during the demonstration.

#### Antenna Setup
Now connect an antenna to the SMA port shown in [Figure 4](#fig-4) on the HW_XM_655 card.

<a class="anchor" id="fig-4"></a>
<figure>
<img src='images/zcu208_adc_setup_antenna.jpg' height='50%' width='50%'/>
    <figcaption><b>Figure 4: Connecting an antenna to the HW_XM_655 card.</b></figcaption>
</figure>

This SMA port is named:
* LFB_ADC_02 (10MHz to 1GHz)


### 1.2. Software Setup <a class="anchor" id="software-setup"></a>
The majority of the software we will use in this demonstration is contained inside the RFSoC-SAM open-source library [2]. We only need to run a few code cells to initialise the software environment.

The RFSoC spectrum analyser features are initialised using `rfsoc_sam.overlay()`. During initialisation the class downloads the spectrum analyser bitstream to the PL and configures the RF Data Converters (DCs) and FPGA IP cores in the system. This process can take around a minute to complete.

Execute the code cell below to load the RFSoC-SAM overlay class.

In [ ]:
from rfsoc_sam.overlay import Overlay

sam = Overlay()

We will now obtain a handle to the spectrum analyser connected to the ADC. This will make it easier to configure the spectrum analyser for the remainder of this notebook. **Please allow a minute for the spectrum analyser to initialise**.

In [ ]:
adc = sam.radio.receiver.channel_12.frontend

## 2. RFSoC Spectral Analysis <a class="anchor" id="rfsoc-spectral-analysis"></a>
We can create a plot to display the captured radio spectrum by running the code cell below.

In [ ]:
adc.analyser.spectrum()

* _**After you have executed the above cell, you should right click the output plot and select "Create New View for Output" from the drop-down menu. This will allow you to interact with the notebook and inspect the spectrum plot in another output view.**_

We will now start the RFSoC spectrum analyser by running the following code cell.

In [ ]:
adc.config = {'spectrum_enable' : True}

The spectrum plot should now be repeatably updating over time. You should be able to see some power in the spectrum at various frequencies. You will also notice the use of the unit 'B' to represent radio frequencies along the x-axis. This unit is the default used by the Python library Plotly, which is used to plot the spectrum (and cannot be changed at this time). Ideally, 'B' should be used to represent GHz, instead.

## 3. Exploring the Spectrum <a class="anchor" id="exploring-the-spectrum"></a>
The radio frequency spectrum is used by many applications. For instance, media and broadcast industries require spectrum to transmit and receive audio and video data. Mobile communications also require radio spectrum to operate correctly. A list is given in Table 1 that contains the frequency range of several applications that require spectrum.

<a class="anchor" id="tab-1"></a>
<figure>
    <figcaption><b>Table 1: A list of frequency bands that we will explore in this notebook.</b></figcaption>
    <br>
    <table style="width:100%">
      <tr>
        <th>Frequency Range</th>
        <th>Description</th>
      <tr style="text-align:center">
        <td>(88 to 108) MHz</td>
        <td>Frequency Modulated (FM) radio broadcasts.</td>
      </tr>
      <tr style="text-align:center">
        <td>(210 to 230) MHz</td>
        <td>Digital Audio Broadcast (DAB) radio communications.</td>
      </tr>
      <tr style="text-align:center">
        <td>(470 to 700) MHz</td>
        <td>Ultra-High Frequency (UHF) digital television broadcasts.</td>
      </tr>
      <tr style="text-align:center">
        <td>(700 to 1000) MHz</td>
        <td>Spectrum access for mobile communications.</td>
      </tr>
      <tr style="text-align:center">
        <td>(1700 to 2200) MHz</td>
        <td>Spectrum access for mobile communications.</td>
      </tr>
    </table>
</figure>

In the following sections, we will use the RFSoC to inspect each of the radio channels above. Many of the frequency ranges above are approximate bands in the spectrum where you should be able to find radio activity in your geographical area. If you are reading this notebook without access to an RFSoC device, then you can see examples of the spectrum given below.

### 3.1. Frequency Modulated Radio <a class="anchor" id="frequency-modulated-radio"></a>
FM radio is typically found across the globe in the frequency range 88MHz to 108MHz. Run the cell below to configure the RFSoC spectrum analyser for FM radio.

In [ ]:
adc.config = {'centre_frequency'  : 98,
              'decimation_factor' : 128}

You should be able to see thin lobes in the spectrum, similar to that shown in Figure 5. These lobes are generally FM radio broadcasts that are used to distribute audio data around a region. FM radio has a bandwidth of approximately 200kHz, which allows 100 different channels to operate at the same time.

<figure>
<img src='images/fm_radio_channels.png' height='75%' width='75%'/>
    <figcaption><b>Figure 5: An example FM radio spectrum.</b></figcaption>
</figure>

### 3.2. Digital Audio Broadcast <a class="anchor" id="digital-audio-broadcast"></a>
Digital Audio Broadcast, or DAB, is not available in every part of the world. The United Kingdom (UK) uses DAB for entertainment radio stations (similar to FM radio). DAB radio channels can vary in bandwidth between regions, but generally operate in the frequency range 210MHz to 230MHz. Run the cell below to configure the spectrum analyser to inspect the DAB frequency bands.

In [ ]:
adc.config = {'centre_frequency'  : 220,
              'decimation_factor' : 128}

Don't worry if you are unable to see any activity in this band. DAB may not be available in your geographical area. You can inspect the spectrum in Figure 6 for an example of DAB radio transmissions.

<figure>
<img src='images/dab_radio_channels.png' height='75%' width='75%'/>
    <figcaption><b>Figure 6: An example DAB radio spectrum.</b></figcaption>
</figure>

### 3.3. Ultra-High Frequency Digital Television <a class="anchor" id="uhf-digital-tv"></a>
The Ultra-High Frequency (UHF) digital TV band is a little difficult to observe as most regions have their own frequency bands for operation. Generally, digital TV broadcasts will be in the frequency range 470MHz to 700MHz. Execute the following code cell to configure the RFSoC to analyse the digital TV spectrum.

In [ ]:
adc.config = {'centre_frequency'  : 595,
              'decimation_factor' : 16,
              'number_frames'     : 64}

It may be a little challenging to see activity in this frequency band. If you live next to a digital TV transmitter you will be able to see several frequency bands in operation and the spectrum will look similar to DAB. However, if you are far away from a transmitter, you may be able to make out some activity. See Figure 7 for the digital TV spectrum captured in Scotland, UK. It is possible to make out several transmission in this capture of the spectrum. However, most are low-power as the spectrum was captured far away from the digital TV transmitter.

<figure>
<img src='images/digital_tv_uhf.png' height='75%' width='75%'/>
    <figcaption><b>Figure 7: An example of digital TV spectrum.</b></figcaption>
</figure>

### 3.4. Spectrum Access for Mobile Communications <a class="anchor" id="spectrum-mobile"></a>
Our last stop in exploring the spectrum is mobile communications. Due to the wide distribution of mobile cellular technology across the globe, it is very easy to inspect and capture this spectrum across many geographical regions. We will start by configuring the RFSoC to analyse the spectrum in the frequency range 700MHz to 1GHz.

In [ ]:
adc.config = {'centre_frequency'  : 850,
              'decimation_factor' : 16}

You are very likely to see activity in this region of the spectrum. Many mobile channels will consume between 5MHz and 20MHz of bandwidth. You can see an example of mobile spectrum captured in the Scotland, UK in Figure 8.

<figure>
<img src='images/spectrum_access_downlink.png' height='75%' width='75%'/>
    <figcaption><b>Figure 8: An example of downlink radio spectrum in the UK around 800MHz.</b></figcaption>
</figure>

There are several frequency bands that have been allocated for mobile communications. Another frequency range exists between 1700MHz and 2200MHz. We can configure the RFSoC to this range by running the code cell below.

In [ ]:
adc.config = {'centre_frequency'  : 1800,
              'decimation_factor' : 16}

You may be able to see transmissions in this part of the spectrum (depending on your geographical location). If not, you can inspect the spectrum captured in Figure 9.

<figure>
<img src='images/spectrum_access_downlink_2GHz.png' height='75%' width='75%'/>
    <figcaption><b>Figure 9: An example of downlink radio spectrum in the UK around 1800MHz.</b></figcaption>
</figure>

## 4. Spectrum Regulation <a class="anchor" id="spectrum-regulation"></a>
We will briefly discuss spectrum regulation and where you may be able to find more information on radio stations that operate in your area.

Most geographical regions in the world have a spectrum regulator that decide how the radio spectrum will be used by organisations and individuals. For example, the spectrum regulator for the UK is the Office of Communications (Ofcom) [3] and the regulator for the United States (US) is the Federal Communications Commission (FCC) [4]. By consulting your spectrum regulator for your geographical region, you will be able to better understand the ambient radio environment.

## 5. Conclusion <a class="anchor" id="conclusion"></a>
This notebook has explored the radio spectrum with the Zynq UltraScale+ RFSoC device. We launched the RFSoC spectrum analyser module and visualised different frequency bands.

In the next notebook, we introduce the RFSoC spectrum analyser application, which you will be able to independently configure to explore the spectrum further with RFSoC.

---

[⬅️ Previous Notebook](../notebook_B/04_digital_filter_design.ipynb) || [Next Notebook 🚀](02_rfsoc_spectrum_analyser.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---